## MyHomeBudget

Aplikacja MyFinance wykorzystuje `TabLayout Navigation` z `ViewPager2`. Dane dostarczymy przez obiekt `dataProvider` zawierający hardcodowane przykładowe pozycje (*dummy data*). Aplikacja zawiera trzy ekrany pokazujące stan oszczędności oraz rachunki - ekran główny zawiera podsumowanie. Dodamy customową animację do `TabLayout.Tab` oraz `DonutChart` podsumowujący stan naszych finansów oraz kilka elementów urozmaicających wygląd naszej aplikacji. Sam layout aplikacji bazuje na przykładzie z **Android Open Project** [link](https://github.com/android/compose-samples/tree/main/Rally) - będzie to wersja aplikacji wcześniej zaimplementowanej w module 1a

<table><tr><td><img src="https://media1.giphy.com/media/49RzyXBmRBa59Wi47n/giphy.gif?cid=790b76111b9b9991bd4197bf8d7295e517603d04f8ba2992&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media1.giphy.com/media/zbqykvBYncTRQLCAja/giphy.gif?cid=790b7611d47e3d6d46e7b736ac29fe3cc9bc281aed07c1c0&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media3.giphy.com/media/BbcDIUyV6IodX2yGVx/giphy.gif?cid=790b7611d1bf2abf7deacc8ad5a71a6a13adfbc9f4344954&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

Rozpocznijmy od dodania zależności do projektu - wykorzystamy `Navigation` oraz będziemy potrzebować dodatkowe ikony w projekcie

In [ ]:
implementation 'androidx.navigation:navigation-compose:2.5.3'
implementation "androidx.compose.material:material-icons-extended:1.4.3"

Nasza aplikacja będzie posiadała zdefiniowany layout tylko w orientacji `portrait`, więc zablookujemy możliwość zmiany w plik `AndroidManifest.xml` - do aktywności dodamy

In [ ]:
android:screenOrientation="portrait"

## Model

Dodajmy model danych

In [ ]:
data class Account(
    val name: String,
    val number: String,
    val amount: Double,
    val color: Int
)

In [ ]:
data class Bill (
    val name: String,
    val endDate: LocalDate,
    val amount: Double,
    val color: Int
)

Dane, podobnie jak w poprzednich przykładach, będziemy pobierać z obiektu `DataProvider`

In [ ]:
object DataProvider {

    val accounts = listOf(
        Account("Home savings", "1111111111", 23456.34, Color.BLUE),
        Account("Car savings", "2222222222", 126578.99, Color.LTGRAY),
        Account("Vacation", "3457733323", 9875.12, Color.MAGENTA),
        Account("Emergency", "9488344443", 10000.77, Color.RED),
        Account("Healthcare", "3243554434", 12345.00, Color.YELLOW),
        Account("Shopping", "2947560007", 3456.56, Color.BLACK)
    )

    val bills = listOf(
        Bill("Bank Credit", LocalDate.of(2022, 9,22), 2300.0, Color.BLACK),
        Bill("Tuition", LocalDate.of(2023, 2,10), 1200.0, Color.BLUE),
        Bill("Rent", LocalDate.of(2022, 8,3), 1023.87, Color.YELLOW),
        Bill("Loan", LocalDate.of(2022, 12,22), 334.0, Color.GRAY),
        Bill("Car Repair", LocalDate.of(2023, 1,9), 982.38, Color.WHITE),
        Bill("Dress Loan", LocalDate.of(2023, 5,18), 243.0, Color.MAGENTA)
    )

    val totalAccountsAmount = (accounts.indices).sumOf { accounts[it].amount }
    val totalBillsAmount = (bills.indices).sumOf { bills[it].amount }
}

## Nawigacja

Nasz aplikacja składa się z trzech ekranów, dodajmy podstawowy layout

In [ ]:
@Composable
fun AccountsScreen(){
    Box(
        Modifier
            .fillMaxSize(),
        contentAlignment = Alignment.Center
    ) {
        Text(
            text = "First Screen",
            fontSize = 40.sp
        )
    }
}

@Preview(showBackground = true)
@Composable
fun FirstPreview() {
    MyHomeBudgetComposeTheme {
        AccountsScreen()
    }
}

In [ ]:
@Composable
fun BillsScreen(){
    Box(
        Modifier
            .fillMaxSize(),
        contentAlignment = Alignment.Center
    ) {
        Text(
            text = "Second Screen",
            fontSize = 40.sp
        )
    }
}

@Preview(showBackground = true)
@Composable
fun SecondPreview() {
    MyHomeBudgetComposeTheme {
        BillsScreen()
    }
}

In [ ]:
@Composable
fun OverviewScreen(){
    Box(
        Modifier
            .fillMaxSize(),
        contentAlignment = Alignment.Center
    ) {
        Text(
            text = "Home Screen",
            fontSize = 40.sp
        )
    }
}

@Preview(showBackground = true)
@Composable
fun HomePreview() {
    MyHomeBudgetComposeTheme {
        OverviewScreen()
    }
}

Dodajmy klasę reprezentującą właściwości ekranów

In [ ]:
sealed class Screens(
    val route: String,
    val icon: ImageVector
) {
    object Overview : Screens("overview", Icons.Filled.AccountBalance)
    object Accounts : Screens("accounts", Icons.Filled.AttachMoney)
    object Bills : Screens("bills", Icons.Filled.MoneyOff)
}

Następnie dodajmy komponenty składowe naszych zakładek, które umieścimy w `topBar` komponentu `Scaffold`. Rozpocznijmy od dodania funkcji pozwalającej na renderowanie pojedynczej zakładki w interfejsie użytkownika. Zakładka może być wybrana lub nieaktywna, a tekst i ikona będą animowane - animacja będzie wyświetlana w zależności od jej stanu.

In [ ]:
@Composable
private fun SingleTab(
    text: String,
    icon: ImageVector,
    onSelected: () -> Unit,
    selected: Boolean
) {}


Funkcja przyjmuje następujące parametry:
- `text` - tekst wyświetlany na zakładce,
- `icon` - ikona wyświetlana na zakładce,
- `onSelected` - funkcja wywoływana po wybraniu zakładki,
- `selected` - flaga wskazująca, czy zakładka jest wybrana.

In [ ]:
@Composable
private fun SingleTab(
    text: String,
    icon: ImageVector,
    onSelected: () -> Unit,
    selected: Boolean
) {

    val color = MaterialTheme.colorScheme.secondary
    val durationMillis = if (selected) TabFadeInAnimationDuration else TabFadeOutAnimationDuration
    val animSpec = remember {
        tween<Color>(
            durationMillis = durationMillis,
            easing = LinearEasing,
            delayMillis = TabFadeInAnimationDelay
        )
    }
    val tabTintColor by animateColorAsState(
        targetValue = if (selected) color else color.copy(alpha = InactiveTabOpacity),
        animationSpec = animSpec
    )
}

private val TabHeight = 56.dp
private const val InactiveTabOpacity = 0.60f

private const val TabFadeInAnimationDuration = 150
private const val TabFadeInAnimationDelay = 100
private const val TabFadeOutAnimationDuration = 100

Definijuemy kilka stałych:
- `color` - kolor używany do tła i tekstu zakładki, pobierany z `MaterialTheme`,
- `durationMillis` - czas trwania animacji dla zakładki, zależny od stanu wybrania zakładki,
- `animSpec` - specyfikacja animacji używanej do animowania koloru zakładki,
- `tabTintColor` - kolor tła i tekstu zakładki, który jest animowany na podstawie stanu wybrania zakładki.

W stałej `animSpec` przechowujemy specyfikację animacji, która zostanie użyta do animowania koloru zakładki. Dzięki zastosowaniu `remember`, specyfikacja animacji jest zapamiętana i nie jest obliczana przy każdym ponownym renderowaniu komponentu. Wewnątrz bloku `remember` tworzymy instancję `tween<Color>`, która jest specyfikacją animacji, działającą na wartości koloru. `tween` to funkcja, która tworzy animację opartą na interpolacji. Przyjmuje ona kilka parametrów:
- `durationMillis` - czas trwania animacji w milisekundach,
- `easing` - funkcja określająca sposób interpolacji wartości animacji (np. `LinearEasing` dla liniowej interpolacji),
- `delayMillis` - opóźnienie animacji przed jej rozpoczęciem w milisekundach.

Gdy zostanie uruchomiona animacja zmiany koloru zakładki, wykorzystamy `animSpec` do kontrolowania jej przebiegu.

Następnie dodajemy `tabColorTint`, który umożliwia animowanie koloru zakładki na podstawie wartości `selected`. Dzięki użyciu `animateColorAsState` i odpowiedniej specyfikacji animacji (`animSpec`), kolor zakładki płynnie zmienia się wraz ze zmianą stanu. 

`animateColorAsState` to funkcja, tworząca stan (`State<Color>`), który może być animowany. Przyjmuje ona dwa parametry:
- `targetValue` - wartość docelowa, do której kolor ma być animowany. Jeśli `selected` ma wartość `true`, kolor zostanie ustawiony na `color` (`MaterialTheme.colorScheme.secondary`), w przeciwnym razie zostanie użyta kopia `color` z zastosowanym zmniejszeniem przez `InactiveTabOpacity` (wartość przezroczystości dla nieaktywnej zakładki)
- `animationSpec` - specyfikacja animacji, która określa czas trwania, opóźnienie i sposób interpolacji animacji.

`tabTintColor` jest wartością zwracaną typu `State<Color>`, który reprezentuje bieżącą wartość koloru zakładki. Kiedy wartość `targetValue` zmienia się na skutek animacji, `tabTintColor` również się zmienia, co powoduje ponowne renderowanie komponentu.

W komponencie używamy `Row`, który definiuje zawartość i wygląd pojedynczej zakładki.

In [ ]:
@Composable
private fun SingleTab(
    text: String,
    icon: ImageVector,
    onSelected: () -> Unit,
    selected: Boolean
) {

    val color = MaterialTheme.colorScheme.secondary
    val durationMillis = if (selected) TabFadeInAnimationDuration else TabFadeOutAnimationDuration
    val animSpec = remember {
        tween<Color>(
            durationMillis = durationMillis,
            easing = LinearEasing,
            delayMillis = TabFadeInAnimationDelay
        )
    }
    val tabTintColor by animateColorAsState(
        targetValue = if (selected) color else color.copy(alpha = InactiveTabOpacity),
        animationSpec = animSpec
    )
    Row(
        modifier = Modifier
            .padding(16.dp)
            .animateContentSize()
            .height(TabHeight)
            .selectable(
                selected = selected,
                onClick = onSelected,
                role = Role.Tab,
                interactionSource = remember { MutableInteractionSource() },
                indication = rememberRipple(
                    bounded = false,
                    radius = Dp.Unspecified,
                    color = Color.Unspecified
                )
            )
            .clearAndSetSemantics { contentDescription = text },
        content =  {
            Icon(imageVector = icon, contentDescription = text, tint = tabTintColor)
            if (selected) {
                Spacer(Modifier.width(12.dp))
                Text(text.uppercase(Locale.getDefault()), color = tabTintColor)
            }
        }
    )
}

private val TabHeight = 56.dp
private const val InactiveTabOpacity = 0.60f

private const val TabFadeInAnimationDuration = 150
private const val TabFadeInAnimationDelay = 100
private const val TabFadeOutAnimationDuration = 100

W modyfikatorze `Row` `animateContentSize()` animuje zmianę rozmiaru wiersza na podstawie zawartości, `selectable` to modifikator, który umożliwia zaznaczanie wiersza. Przyjmuje kilka parametrów:
- `selected` - wartość logiczna, czy dany wiersz jest zaznaczony czy nie,

- `onClick` - funkcja wywoływana po kliknięciu wiersza, w tym przypadku wywołuje funkcję `onSelected`, która jest przekazywana jako parametr,

- `role` - rola wiersza, w tym przypadku jest to rola `Role.Tab`, co wskazuje na to, że wiersz reprezentuje zakładkę - można nadać zakładce wiele różnych ról: 
    - `Role.Tab` - rola oznaczająca, że wiersz reprezentuje zakładkę.
    - `Role.Button` - rola oznaczająca, że wiersz reprezentuje przycisk.
    - `Role.Checkbox` - rola oznaczająca, że wiersz reprezentuje pole wyboru (*checkbox*).
    - `Role.RadioButton` - rola oznaczająca, że wiersz reprezentuje przycisk opcji (*radio button*).
<br/><br/>
 
- `interactionSource` - źródło interakcji, w tym przypadku jest to nowa instancja `MutableInteractionSource` - jest to źródło interakcji, które może być używane do rejestrowania i odczytywania zdarzeń interakcji. Przechowuje informacje o bieżących stanach interakcji, takich jak naciśnięcie, dotknięcie, przeciągnięcie, wybór itp. Dzięki `MutableInteractionSource` możemy monitorować i reagować na różne zdarzenia interakcji w naszych komponentach Compose. Możemy przekazać MutableInteractionSource do odpowiednich komponentów, takich jak `Clickable`, `Selectable`, `Draggable`, aby obsłużyć i reagować na interakcje użytkownika. Na przykład, możemy śledzić kliknięcia na wiersz, monitorować stan naciśnięcia lub odznaczenia w polu wyboru, itp.

- `indication` - efekt wizualny podczas interakcji z wierszem, w tym przypadku używamy efektu *ripple* (falowania) za pomocą funkcji `rememberRipple`. Funkcja ta zwraca obiekt `RippleIndication` zawierający informacje o zdefiniowanym efekcie falowania. Następnie przekazujemy ten obiekt do modifikatora `selectable`.
    - `bounded` Określa, czy efekt falowania ma być ograniczony do granic wiersza (`true`), czy ma wypełnić całą przestrzeń wokół wiersza (`false`).
    - `radius` Określa promień efektu falowania. Może przyjąć wartość `Dp.Unspecified` dla dynamicznego określenia promienia na podstawie rozmiaru wiersza.
    - `color` Określa kolor efektu falowania. Może być używany kolor zdefiniowany wcześniej lub `Color.Unspecified` dla automatycznego dopasowania koloru do motywu.
    <br/><br/>
- `clearAndSetSemantics` to modifikator, który ustawia opis treści (semantykę) dostępności dla wiersza na wartość `text`, która jest przekazywana jako parametr. Jest używane głównie w celu dostarczenia informacji o dostępności i ułatwienia korzystania z interfejsu użytkownika dla osób korzystających z technologii asystujących.
    - Opisy obrazków: Możemy użyć do ustawienia opisu (`contentDescription`) dla obrazków w celu opisania ich zawartości osobom niewidomym lub słabowidzącym.
    - Tekst alternatywny dla elementów interaktywnych: Możemy ustawić tekst alternatywny (`contentDescription`) dla elementów interaktywnych, takich jak przyciski lub linki, aby przekazać informację o ich funkcji użytkownikom, którzy nie widzą lub mają trudności z odczytaniem etykiet.
    - Dostępność treści: Możemy użyć do dostarczenia informacji o dostępności dla różnych elementów treści, takich jak nagłówki, akapity, listy, tabelki, itp.
    </br></br>
    
- `content` - zawartość wiersza
    - `Icon` reprezentuje ikonę zakładki. Przyjmuje trzy parametry:
        - `imageVector` Obraz wektorowy (np. ikona), który ma być wyświetlany jako ikona zakładki.
        - `contentDescription` Opis treści ikony, który jest przekazywany w celach dostępności.
        - `tint` Kolor, który ma być zastosowany do ikony zakładki.
        </br></br>
    - Warunek `if (selected)` Jest to warunek sprawdzający, czy dany wiersz jest zaznaczony. Jeśli tak, to wyświetlany jest dodatkowy element wiersza.
    - `Spacer(Modifier.width(12.dp))` Komponent używany jest do utworzenia pustej przestrzeni pomiędzy ikoną a tekstem.
    - `Text` Ten komponent reprezentuje tekst zakładki. Przyjmuje dwa parametry:
        - `text` Treść, która ma być wyświetlana jako tekst zakładki.
        - `color` Kolor tekstu zakładki.
    </br></br>
    Oznacza to, że jeśli wiersz jest zaznaczony (selected), zostanie wyświetlony tekst zakładki obok ikony, z odpowiednimi odstępami i kolorem tekstu zdefiniowanym przez tabTintColor. Jeśli wiersz nie jest zaznaczony, tylko ikona zakładki będzie widoczna.

Zdefiniujmy komponent reprezentujący wiersz z zakładkami.

In [ ]:
@Composable
fun TabRow(
    allScreens: List<Screens>,
    onTabSelected: (Screens) -> Unit,
    currentScreen: Screens
) {
    Surface(
        Modifier
            .height(TabHeight)
            .fillMaxWidth()
    ) {
        Row(Modifier.selectableGroup()) {
            allScreens.forEach { screen ->
                SingleTab(
                    text = screen.route,
                    icon = screen.icon,
                    onSelected = { onTabSelected(screen) },
                    selected = currentScreen == screen
                )
            }
        }
    }
}

Komponent ten przyjmuje trzy parametry:
- `allScreens` Lista wszystkich dostępnych zakładek (obiektów typu `Screens`).
- `onTabSelected` Funkcja, która jest wywoływana po wybraniu zakładki. Przyjmuje jako argument wybraną zakładkę.
- `currentScreen` Obecnie wybrana zakładka.


Komponent `TabRow` tworzy wiersz zakładek używając komponentów `Surface` i `Row`. Wewnątrz wiersza iteruje po wszystkich zakładkach z listy `allScreens` i dla każdej zakładki renderuje komponent `SingleTab`. `Surface` tworzy powierzchnię, która otacza wiersz zakładek. `Row` tworzy poziomy układ, w którym są renderowane poszczególne zakładki. Używa on również modyfikatora `selectableGroup()`, który umożliwia zaznaczanie jednego elementu wiersza. `onSelected` to funkcja, która jest wywoływana po wybraniu zakładki. `selected` informuje, czy dana zakładka jest obecnie wybrana (porównuje się ją z `currentScreen`).

W efekcie, komponent `TabRow` renderuje wiersz zakładek, gdzie każda zakładka jest reprezentowana przez komponent `SingleTab`, a wybrana zakładka ma odpowiednie oznaczenie graficzne. Po kliknięciu na zakładkę wywoływana jest funkcja `onTabSelected` z wybraną zakładką jako argumentem.

Następnie tworzymy nawigację, rozpoczniemy od utworzeniu grafu nawigacji.

In [ ]:
@Composable
fun NavGraph(navController: NavHostController, modifier: Modifier){
    NavHost(
        navController = navController,
        startDestination = Screens.Overview.route,
        modifier = modifier
    ) {
        composable(route = Screens.Overview.route){ OverviewScreen() }
        composable(route = Screens.Accounts.route){ AccountsScreen() }
        composable(route = Screens.Bills.route){ BillsScreen() }
    }
}

Aby uniknąć problemu gromadzenia dużej liczby ekranów na stosie nawigacji, w miarę wybierania przez użytkowników elementów, dodajmy funkcję rozszerzającą zapewniającą zachowanie `singleTop`

In [ ]:
fun NavHostController.navigateSingleTopTo(route: String) =
    this.navigate(route) {
        // Pop up to the start destination of the graph to
        // avoid building up a large stack of destinations
        // on the back stack as users select items
        popUpTo(
            this@navigateSingleTopTo.graph.findStartDestination().id
        ) {
            saveState = true
        }
        // Avoid multiple copies of the same destination when
        // reselecting the same item
        launchSingleTop = true
        // Restore state when reselecting a previously selected item
        restoreState = true
}

Przed wywołaniem nawigacji, zdefiniowane są pewne ustawienia nawigacji typu:
- `popUpTo` Ustala, że należy cofnąć się do startowego celu grafu, aby uniknąć gromadzenia dużej ich liczby na stosie nawigacji. W przypadku nawigacji do nowego celu, stare zostaną usunięte z powrotem do punktu startowego. Ta konfiguracja jest osiągana poprzez wywołanie metody `popUpTo` i przekazanie identyfikatora startowego celu grafu (odnaleziony poprzez `this@navigateSingleTopTo.graph.findStartDestination().id`).
- `launchSingleTop` Ustala, że nawigacja do tego samego celiu, która jest już wybrana, nie spowoduje tworzenia nowej instancji tego celu. Zamiast tego, zostanie przywrócona już istniejąca. Ta konfiguracja jest osiągana przez ustawienie wartości `launchSingleTop` na `true`.
- `restoreState` Ustala, że stan poprzednio wybranego celu zostanie przywrócony, jeśli ten sam cel zostanie ponownie wybrany. W efekcie, użytkownikowi zostanie przedstawiony ostatni stan tego celu, a nie będzie konieczności budowania nowego stanu. Ta konfiguracja jest osiągana przez ustawienie wartości `restoreState` na `true`.

Ostatnią funkcję w tej części jest `Navigation`, odpowiedzialna za nawigację.

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun Navigation(){
    val navController = rememberNavController()
    val screens = listOf(
        Screens.Overview, Screens.Accounts, Screens.Bills
    )

    val navBackStackEntry by navController.currentBackStackEntryAsState()
    val currentDestination = navBackStackEntry?.destination
    val currentScreen =
        screens.find { it.route == currentDestination?.route } ?: Screens.Overview

    Scaffold(
        topBar = {
            TabRow(
                allScreens = screens,
                onTabSelected = { newScreen ->
                    navController.navigateSingleTopTo(newScreen.route)
                },
                currentScreen = currentScreen
            )
        }
    ) { paddingValues ->
        NavGraph(
            navController = navController,
            modifier = Modifier.padding(paddingValues)
        )
    }
}

Bieżący ekran (`currentScreen`) odnajdujemy na podstawie wartości `currentDestination` i listy `screens`. Jeżeli nie zostanie znaleziony pasujący ekran, przyjmuje się ekran startowy (`Screens.Overview`). Następnie renderowany jest `Scaffold` z górnym paskiem (`topBar`) zawierającym `TabRow`. Otrzymuje on listę wszystkich dostępnych ekranów (`screens`), funkcję obsługującą zmianę wybranego ekranu (`onTabSelected`) oraz informację o bieżącym ekranie (`currentScreen`). Przy zmianie ekranu wykorzystywana jest funkcja `navController.navigateSingleTopTo()`, która odpowiada za nawigację do nowego ekranu w trybie *single top* - czyli cofnięcie się do punktu startowego nawigacji i uniknięcie duplikacji na stosie nawigacji. Na koniec renderowany jest `NavGraph`, który obsługuje resztę nawigacji w aplikacji. Przekazany jest do niego `navController` oraz `paddingValues` w celu odpowiedniego wcięcia treści w obszarze głównym (`content`) `Scaffold`.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExODA2MDVlNDU5M2Y5MzdlNWQyNjZlODE0YTRjMWRlNDBiYjgzNmVjNSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/1wWu0Rv6XAYaWhHmQr/giphy.gif" width="200" />